In [2]:
from MaxAgents import MaxRandomAgent, MaxGrimTriggerAgent
from MinAgents import MinRandomAgent
from Model import MyModel, CustomDataCollector
from math import floor

## Parameters 
N = 10
prop_adversaries = .3
coin_toss = prop_adversaries
rounds_factor = 3

model = MyModel(N, coin_toss, prop_adversaries, MaxGrimTriggerAgent, MinRandomAgent)

for i in range(floor(N * rounds_factor)):
    model.step()

df = CustomDataCollector(model)

/Users/jho/anaconda3/lib/python3.10/site-packages/mesa/time.py:82: FutureWarning: The AgentSet is experimental. It may be changed or removed in any and all future releases, including patch releases.
We would love to hear what you think about this new feature. If you have any thoughts, share them with us here: https://github.com/projectmesa/mesa/discussions/1919
  self._agents: AgentSet = AgentSet(agents, model)


In [4]:
model.check_historical_choose_chosen_pairs()

[(0, 4),
 (1, 2),
 (2, 5),
 (3, 5),
 (4, 9),
 (5, 8),
 (6, 7),
 (7, 0),
 (8, 0),
 (9, 7),
 (0, 6),
 (1, 6),
 (2, 5),
 (3, 7),
 (4, 7),
 (5, 3),
 (6, 0),
 (7, 0),
 (8, 7),
 (9, 8),
 (0, 8),
 (1, 3),
 (2, 6),
 (3, 4),
 (4, 0),
 (5, 8),
 (6, 4),
 (7, 6),
 (8, 6),
 (9, 6)]

In [9]:
#model.check_historical_action_outcome(MinRandomAgent)
model.check_adversarial(MinRandomAgent)

[False, True, True, False, False, True, False, False, False, False]

In [2]:
sum(df["Outcome"]) / floor(N * rounds_factor)

0.8

In [3]:
df.head()

,Pairs,Voting,Outcome,Action
0,"(0, 6)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,"[1, 1]"
1,"(1, 3)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,"[1, 1]"
2,"(2, 5)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,"[1, 0]"
3,"(3, 9)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",1,"[1, 1]"
4,"(4, 3)","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]",0,"[0, 1]"


In [4]:
beliefs = []
adversarial = model.adversarial
for i in range(0, len(adversarial)):
    beliefs.append(model.agents[i].belief)
    print(model.agents[i].id)

0
1
2
3
6
8
9
4
5
7


In [5]:
adversarial

[False, False, False, False, True, True, False, True, False, False]

In [ ]:
beliefs

In [ ]:
df.head(20)

In [1]:
min([1,1,2,4])

1